In [2]:
%matplotlib inline


## הקדמה בסיסית `torch.autograd`

`torch.autograd` הוא מנוע בידול? אוטומטי שמאפשר לאמן רשתות נוירונים. בפרק הזה ננסה להסביר בצורה כללית את איך `torch.autograd` מאפשר לאמן רשתות נויירונים
### טנזורים ורשת נוירונים _nerual networks_
> רשת נוירונים  _nerual networks (NN)_ היא קבוצה של פונקציות משורשרות אחת בתוך השניה, שמפעילים אותה על מידע כלשהוא ע"מ לחזות את העתיד (ע"פ רוב).  אותם פונקציות מוגדרות על ידי פרמטרים שנקראים _biases_ או _weights_ שאותם _PyTorch_  מאחסנת בטנזורים. 

ע"מ שNN יפעל כמו שצריך, כלומר יהיה לנו את ה_weights_ (משקולות) הנכונים, צריך לאמן את הרשת.
ע"מ לאמן את הרשת צריך שיהיה לנו מידע _input_ ותוצאות שידוע שהם נכונות ? ואז ניתן לעשות את שלב האימון:

**פיעפוע לפנים? _Forward Propagation_**
בתהליך זה האלגוריתם לוקח את המשקולות הנתונים לו (בצעד הראשון אלו משקולות אקראיים) ומחשב את התוצאה עבור מידע מסויים. 

**פיעפוע לאחר _Backward Propagation_**
בתהליך זה האלגוריתם משווה את התוצאה עם התוצאה הידועה לו כנכונה, ומעדכן את המשקולות באופן פפרציונלי לשגיאה. הוא עושה את זה ע"י מעבר אחורה על הפונקציות ושינוי של הפרמטרים שלהם ע"מ שיתקרבו לתשובה הנכונה - ע"י מבט על הנגזרת שלהם. פונקציה זאת נקראת _gradient descent_. ע"מ להבין טוב יותר ניתן לראות את הסרטון [הזה](https://www.youtube.com/watch?v=tIeHLnjs5U8) של _3Blue1Brown_  



## אימון עם _PyTorch_


נתבונן על מחזור אימון אחד. 
בדוגמה הזאת נטען מודל מאומן מראש בשם `resnet18` מתוך חבילת המודלים המאומנים `torchvision`. אנחנו ניצור טנזור רנדומלי `data` שייצג תמונה עם 3 ערוצים (RBG), הגובה והרוחב של התמונה הם 64X64 פיקסלים.
לתמונה האקראית ניצור גם `label` אקראי שייצג את מה שיש בתמונה. הצורה של המודלים היא (1,1000) ??

In [3]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64) #ramdom image
labels = torch.rand(1, 1000)

נוכל לראות את כל שכבה ואת המאפיינים שלה ע"י מעבר על `()model.named_parameters` (יש המון מידע בשכבות, אז לא כדאי להדפיס את כל המשקלים

In [4]:
for name, param in model.named_parameters():
    print(name, end = "-> ")
    print(param.requires_grad) #if need to update it in Backward Propagation
#     print(param) #All the weights of the nets it's huge
    

conv1.weight-> True
bn1.weight-> True
bn1.bias-> True
layer1.0.conv1.weight-> True
layer1.0.bn1.weight-> True
layer1.0.bn1.bias-> True
layer1.0.conv2.weight-> True
layer1.0.bn2.weight-> True
layer1.0.bn2.bias-> True
layer1.1.conv1.weight-> True
layer1.1.bn1.weight-> True
layer1.1.bn1.bias-> True
layer1.1.conv2.weight-> True
layer1.1.bn2.weight-> True
layer1.1.bn2.bias-> True
layer2.0.conv1.weight-> True
layer2.0.bn1.weight-> True
layer2.0.bn1.bias-> True
layer2.0.conv2.weight-> True
layer2.0.bn2.weight-> True
layer2.0.bn2.bias-> True
layer2.0.downsample.0.weight-> True
layer2.0.downsample.1.weight-> True
layer2.0.downsample.1.bias-> True
layer2.1.conv1.weight-> True
layer2.1.bn1.weight-> True
layer2.1.bn1.bias-> True
layer2.1.conv2.weight-> True
layer2.1.bn2.weight-> True
layer2.1.bn2.bias-> True
layer3.0.conv1.weight-> True
layer3.0.bn1.weight-> True
layer3.0.bn1.bias-> True
layer3.0.conv2.weight-> True
layer3.0.bn2.weight-> True
layer3.0.bn2.bias-> True
layer3.0.downsample.0.weight->

בשלב הבא נעביר את הקלט (התמונה האקראית שיצרנו) דרך כל אחת משכבות המודל, נקבל בפלט חיזוי מהו תוכן התמונה.
פלט החיזוי הוא טנזור שמכיל לכל ליבל את הסיכוי שזה הוא. 


In [5]:
prediction = model(data) # forward pass

In [6]:
prediction.shape

torch.Size([1, 1000])

נשווה את מה שהמודל חזה ואת מה שידוע לנו שהיא התוצאה הנכונה, נסכום את ההפרש בינהם כך נוכל לחשב את שיעור הטעות `loss`, בשלב הבא נשנה את המשקולות בהתאמה בתהליך הפיעפוע לאחור _backward_ 

תהליך הפיעפוע לאחור מתחיל ע"י קריאה למתודה `()backward.` על הטנזור הטעות `loss`. לאחר מכן בצורה אוטומטית _Autograd_ מחשב ומאחסן את הגרדיאנט _gradients_ לכל מודל פרמטר של המודל בתוך המאפיין (_attribute_) `grad.`  של הפרמטר.  
>💡 שימו לב - המתודה `()backward.` נגשת לכל שכבה במודל ומכניסה לה את ערך ה_grad_, למרות שהיא נקראת על טנזור ה_loss_  


In [7]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

בשלב הבא נפעיל את תהליך האופטימיזציה על המודל, במקרה הזה נשתמש ב_SGD_ כך שהפרמטרים _learninig rate_ `lr` הוא 0.01 והמומנטום [_momentum_]((https://towardsdatascience.com/stochastic-gradient-descent-with-momentum-a84097641a5d) הוא 0.9, נכניס את כל הפרמטרים לתוך האופטימייזר.  


In [8]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

לבסוף קוראים למתודה `()step.` ע"מ להחיל את ה_gradient descent_ על הפרמטרים. האפטימייזר ישנה את הערכים של כל פרמטר ופרמטר לפי הגרינאנט _gradient_ שנמצא ב`grad.`

In [9]:
optim.step() #gradient descent

בשלב הזה סיימנו את כל התהליך של אימון רשת נוירונים. בהמשך הפרק נראה איך הדברים פועלים מבחינה מתמטית. 


--------------
## מה זה גרדיאנט _gradient_





### חישוב אוטומטי של ה`grad.` ע"י _autograd_
נראה כיצד _autograd_  כותב את ערכי ה`grad.`
ניצור שני טנזורים _a_ ו_b_ עם השדה`requires_grad=True` כך נסמן ל_autograd_ שצריך לעקוב אחרי כל השינויים שקוראים בטנזור הזה 

In [1]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

ניצור טנזור אחר _Q_ מ_a_ ו_b_

\begin{align}Q = 3a^3 - b^2\end{align}



In [ ]:
Q = 3*a**3 - b**2

נניח ש_a_ ו_b_ הם פרמטרים של רשת נויירונים כל שהיא ו_Q_ הוא טנזור השגיאה _loss_, באימון הרשת אנחנו רוצים לחשב את גרדיאנט של הפרמטרים, כלומר הנגזרת של הפער בן הטנזור הנוכחי לטנזור השגיאה לפי המשתנים השונים

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}

כאשר אנחנו קוראים למתודה `()backward.` על _Q_ בצורה אוטמטית _autograd_ מחשבת את הגרדיאנט ומאחסנת אותו בתוך המאפיין (_attribute_)  `grad.` 



אנחנו צריכים בצורה מפורשת להעביר טנזור _gradient_ כארגומנט לתוך `()Q.backward`  בגלל שהוא וקטור, הוא מייצג את הגרדיאנט של _Q_, מכיון ביחס לעצמו - ששווה לטנזור אחדות

\begin{align}\frac{dQ}{dQ} = 1\end{align}

לחלופין ניתן לסכום את אברי _Q_ ואז לקרוא ל`()Q.sum().backward` בלי להעביר את הגרדיאנט של השגיאה

In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradients are now deposited in ``a.grad`` and ``b.grad``



In [2]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

False
False


Optional Reading - Vector Calculus using ``autograd``
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Mathematically, if you have a vector valued function
$\vec{y}=f(\vec{x})$, then the gradient of $\vec{y}$ with
respect to $\vec{x}$ is a Jacobian matrix $J$:

\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector $\vec{v}$, compute the product
$J^{T}\cdot \vec{v}$

If $\vec{v}$ happens to be the gradient of a scalar function $l=g\left(\vec{y}\right)$:

\begin{align}\vec{v}
   =
   \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}\end{align}

then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)\end{align}

This characteristic of vector-Jacobian product is what we use in the above example;
``external_grad`` represents $\vec{v}$.




Computational Graph
~~~~~~~~~~~~~~~~~~~

Conceptually, autograd keeps a record of data (tensors) & all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
`Function <https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function>`__
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor, and
- maintain the operation’s *gradient function* in the DAG.

The backward pass kicks off when ``.backward()`` is called on the DAG
root. ``autograd`` then:

- computes the gradients from each ``.grad_fn``,
- accumulates them in the respective tensor’s ``.grad`` attribute, and
- using the chain rule, propagates all the way to the leaf tensors.

Below is a visual representation of the DAG in our example. In the graph,
the arrows are in the direction of the forward pass. The nodes represent the backward functions
of each operation in the forward pass. The leaf nodes in blue represent our leaf tensors ``a`` and ``b``.

.. figure:: /_static/img/dag_autograd.png

<div class="alert alert-info"><h4>Note</h4><p>**DAGs are dynamic in PyTorch**
  An important thing to note is that the graph is recreated from scratch; after each
  ``.backward()`` call, autograd starts populating a new graph. This is
  exactly what allows you to use control flow statements in your model;
  you can change the shape, size and operations at every iteration if
  needed.</p></div>

Exclusion from the DAG
^^^^^^^^^^^^^^^^^^^^^^

``torch.autograd`` tracks operations on all tensors which have their
``requires_grad`` flag set to ``True``. For tensors that don’t require
gradients, setting this attribute to ``False`` excludes it from the
gradient computation DAG.

The output tensor of an operation will require gradients even if only a
single input tensor has ``requires_grad=True``.




In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

In a NN, parameters that don't compute gradients are usually called **frozen parameters**.
It is useful to "freeze" part of your model if you know in advance that you won't need the gradients of those parameters
(this offers some performance benefits by reducing autograd computations).

Another common usecase where exclusion from the DAG is important is for
`finetuning a pretrained network <https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html>`__

In finetuning, we freeze most of the model and typically only modify the classifier layers to make predictions on new labels.
Let's walk through a small example to demonstrate this. As before, we load a pretrained resnet18 model, and freeze all the parameters.



In [ ]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Let's say we want to finetune the model on a new dataset with 10 labels.
In resnet, the classifier is the last linear layer ``model.fc``.
We can simply replace it with a new linear layer (unfrozen by default)
that acts as our classifier.



In [ ]:
model.fc = nn.Linear(512, 10)

Now all parameters in the model, except the parameters of ``model.fc``, are frozen.
The only parameters that compute gradients are the weights and bias of ``model.fc``.



In [ ]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Notice although we register all the parameters in the optimizer,
the only parameters that are computing gradients (and hence updated in gradient descent)
are the weights and bias of the classifier.

The same exclusionary functionality is available as a context manager in
`torch.no_grad() <https://pytorch.org/docs/stable/generated/torch.no_grad.html>`__




--------------




Further readings:
~~~~~~~~~~~~~~~~~~~

-  `In-place operations & Multithreaded Autograd <https://pytorch.org/docs/stable/notes/autograd.html>`__
-  `Example implementation of reverse-mode autodiff <https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC>`__

